# Step 1: fine-tune LLM using top result from (fixed) ranker

In [ ]:
def launch():
    import os
    import StepOne
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    os.environ['BATCH_SIZE'] = '8'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepOne.step_one,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 4 max_iteration = 5 model_type = base *********
n                 iter (since)         4 loss (since)       4 rouge1 (since)   4 rouge1 ema (since) 4 rouge1 (dev) (since)      dt
1                0.000 (0.000)          2.260 (2.260)          0.426 (0.426)          0.426 (0.426)          0.000 (0.000)  7.66 s
2                0.000 (0.000)          2.047 (1.835)          0.464 (0.502)          0.459 (0.492)          0.000 (0.000)  13.9 s
4                0.000 (0.000)          1.949 (1.850)          0.436 (0.409)          0.462 (0.466)          0.000 (0.000)  25.2 s
16               0.000 (0.000)          1.983 (1.987)          0.426 (0.431)          0.435 (0.424)          0.000 (0.000)  1.56 m
32               0.000 (0.000)          1.967 (1.950)          0.438 (0.449)          0.440 (0.444)          0.000 (0.000)  3.01 m
64               0.000 (0.000)          1.902 (1.838)          0.435 (0.433)          0.438 (0.437)          0.000 (0.000)  5.93 m
128             

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def launch():
    import os
    import StepTwo
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    os.environ['BATCH_SIZE'] = '2'
    os.environ['LEARN_BATCH_SIZE'] = '4'
    os.environ['GRAD_FREE_BATCH_SIZE'] = '16'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepTwo.step_two,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 1 max_iteration = 20 model_type = base *********
n                    iter (since)            4 loss (since)           4 rouge (since)       4 ema rouge (since) 4 ema rouge (dev) (since)            nsamps (since)      dt
2                   0.000 (0.000)             0.655 (0.660)             0.407 (0.277)             0.372 (0.206)             0.000 (0.000)            60.500 (6.500)  1.04 m
4                   0.000 (0.000)             0.676 (0.696)             0.457 (0.506)             0.424 (0.476)             0.000 (0.000)            44.625 (28.750)  1.96 m
8                   0.000 (0.000)             0.648 (0.621)             0.480 (0.502)             0.481 (0.538)             0.000 (0.000)            33.000 (21.375)  3.14 m
16                  0.000 (0.000)             0.662 (0.676)             0.466 (0.453)             0.453 (0.426)             0.000 (0.000)            27.688 (22.375)  5.78 m
32                  0.000 (0.000)             0.678 (0.695)          

# Step 3: Prepare Submission Files

In [ ]:
def prepare_submission(*, step2_iter, step1_iter, k):
    import evaluate
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedScholarly import dev_loader, test_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(8675309)

    dev = dev_loader(batch_size=8)
    test = test_loader(batch_size=8)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    taskllm_model_id = f'User_keq{k}_t5base_step1_iter{step1_iter}'
    t5.load_adapter(taskllm_model_id, 'raw_taskllm')
    t5.load_adapter(taskllm_model_id, 'ema_taskllm')
    rhat_model_id = f'User_keq{k}_t5base_step2_iter{step2_iter}'
    t5.load_adapter(rhat_model_id, 'raw_rhat')
    t5.load_adapter(rhat_model_id, 'ema_rhat')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_suffix="taskllm")
    rewardpredictor = RewardPredictor(t5=t5, adapter_suffix="rhat", model_id=rhat_model_id)
    rouge_metric = evaluate.load('rouge')
    gradfree_batch_size = 128
    n_randos = 128

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(embeddings, nsamples):
        scores = embeddings[0,:] @ embeddings[1:,:].T
        temperature = scores[0].item() - scores[min(scores.shape[0]-1, 4)].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape).to(scores.device)
        safek = min(k, scores.shape[0])
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=safek).indices, sorted=False, dim=0)

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]
    
    print(f'*** step1_iter: {step1_iter} step2_iter: {step2_iter} ***')

    devgolds, testgolds = [], []
    with ProgressPrinter(f'{k} rouge (dev)') as printer:
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for isdev, (examples, labels) in interleave(dev, test, sequential=True):
            with torch.no_grad():
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['abstract'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['abstract'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = gradfree_batch_size,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                randos = [ randomized_similarity(embeddings[a:b,:], n_randos) for a, b in zip(splits, splits[1:]) ]
                prompts = [ [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in indices ])
                              for indices in rando.to('cpu').tolist()
                            ]
                            for ex, rando in zip(examples, randos)
                          ]
                rhats = torch.cat(inner_batch(func = rewardpredictor.predict,
                                              inner_batch_size = gradfree_batch_size,
                                              inputs = (sum(prompts, []),)
                                             ),
                                  dim=0)
                splits = cumsum(map(len, prompts))
                greedyaction = [ rhats[a:b].argmax().item() for a, b in zip(splits, splits[1:]) ]
                greedyprompts = [ prompt[a] for prompt, a in zip(prompts, greedyaction) ]
                guesses = sum(inner_batch(func = taskllm.generate,
                                          inner_batch_size = gradfree_batch_size,
                                          inputs = (greedyprompts,)
                                         ),
                              [])
                if isdev:
                    rewards = sum( ( rouge_metric.compute(predictions=[guess],
                                                          references=[label],
                                                          use_aggregator=False)['rouge1']
                                     for guess, label in zip(guesses, labels)
                                  ),
                                  [])
                    rewards = torch.Tensor(rewards, device='cpu').mean().item()
                else:
                    rewards = None
                
                for ex, guess in zip(examples, guesses):
                    (devgolds if isdev else testgolds).append({ 'id': ex['id'], 'output': guess })

            printer.addobs(rewards)

    for wut, golds in ( ('dev', devgolds), ('test', testgolds) ):
        with open(f'lamp5u_{wut}golds_t5base_keq{k}_step1_iter{step1_iter}_step2_iter{step2_iter}.json', 'w') as jsonfile:
            json.dump({ 'task': 'LaMP_5', 'golds': golds }, jsonfile)
            
prepare_submission(k=4, step1_iter='4_augment4', step2_iter='7_augment1')

*** step1_iter: 4_augment4 step2_iter: 7_augment1 ***
n       4 rouge (dev) (since)      dt
1               0.422 (0.422)  7.84 s
2               0.428 (0.434)  16.4 s
4               0.449 (0.469)    36 s
8               0.472 (0.495)  1.21 m
16              0.451 (0.430)  2.44 m
32              0.433 (0.415)  4.87 m
64              0.422 (0.410)  9.68 m
128             0.434 (0.446)  19.1 m
256             0.434 (0.435)  38.1 m
512             0.432 (0.422)  1.27 h
